In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
batchsize=8

In [5]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\19IT042\SEM 6\SGP\Drowsiness detection\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\19IT042\SEM 6\SGP\Drowsiness detection\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 67919 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\19IT042\SEM 6\SGP\Drowsiness detection\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 8306 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_752 (Conv2D)             (None, 39, 39, 32)   864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_752 (BatchN (None, 39, 39, 32)   96          conv2d_752[0][0]                 
__________________________________________________________________________________________________
activation_752 (Activation)     (None, 39, 39, 32)   0           batch_normalization_752[0][0]    
____________________________________________________________________________________________

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [9]:
checkpoint = ModelCheckpoint(r'D:\19IT042\SEM 6\SGP\Drowsiness detection\model\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_10752/3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
8489/8489 [==============================] - ETA: 0s - loss: 0.2016 - accuracy: 0.9215
Epoch 1: val_loss improved from inf to 0.25192, saving model to D:\19IT042\SEM 6\SGP\Drowsiness detection\model\model.h5
8489/8489 [==============================] - 3574s 420ms/step - loss: 0.2016 - accuracy: 0.9215 - val_loss: 0.2519 - val_accuracy: 0.9047 - lr: 0.0010
Epoch 2/5
8489/8489 [==============================] - ETA: 0s - loss: 0.1725 - accuracy: 0.9328
Epoch 2: val_loss improved from 0.25192 to 0.23844, saving model to D:\19IT042\SEM 6\SGP\Drowsiness detection\model\model.h5
8489/8489 [==============================] - 759s 89ms/step - loss: 0.1725 - accuracy: 0.9328 - val_loss: 0.2384 - val_accuracy: 0.9063 - lr: 0.0010
Epoch 3/5
8489/8489 [==============================] - ETA: 0s - loss: 0.1657 - accuracy: 0.9359
Epoch 3: val_loss did not improve from 0.23844
8489/8489 [==============================] - 891s 105ms/step - loss: 0.1657 - accuracy: 0.9359 - val_loss: 0.2865 - 

In [ ]:
# Model Evaluation

In [11]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_10752/713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.13633036613464355
0.9485858082771301


In [12]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_10752/4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.2519475221633911
0.9101242423057556


In [13]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_10752/3655471885.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.13633036613464355
0.9485858082771301
